# Bitcoin Individual News Model Evaluation
This notebook evaluates the Bitcoin prediction model trained on the individual news dataset with longer training parameters and provides comprehensive analysis with base model comparison.

## Install Required Libraries

In [ ]:
# Install required libraries for GRPO training
!pip install trl==0.7.6 transformers>=4.38.0 datasets accelerate bitsandbytes
!pip install wandb torch>=2.0.0 peft>=0.8.0 scipy evaluate
!pip install deepspeed

# Additional libraries for data processing and visualization
!pip install pandas matplotlib seaborn numpy

## Troubleshooting: Environment Setup

If you encounter the error `operator torchvision::nms does not exist`, this is due to version incompatibility between PyTorch, torchvision, and transformers. The installation cell above installs compatible versions.

**Alternative solutions if the error persists:**

1. **Create a fresh environment:**
   ```bash
   conda create -n bitcoin_eval python=3.10
   conda activate bitcoin_eval
   ```

2. **Install in specific order:**
   ```bash
   pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0
   pip install transformers==4.35.0
   pip install datasets peft accelerate
   ```

3. **If using CPU only:**
   ```bash
   pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cpu
   ```

**Note:** After installation, restart your kernel before proceeding to the next cell.

In [ ]:
# Verify installation and check for compatibility issues
import sys
print(f"Python version: {sys.version}")

try:
    import torch
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA version: {torch.version.cuda}")
        print(f"GPU count: {torch.cuda.device_count()}")
except ImportError as e:
    print(f"PyTorch import error: {e}")

try:
    import torchvision
    print(f"Torchvision version: {torchvision.__version__}")
except ImportError as e:
    print(f"Torchvision import error: {e}")

try:
    import transformers
    print(f"Transformers version: {transformers.__version__}")
except ImportError as e:
    print(f"Transformers import error: {e}")

try:
    import peft
    print(f"PEFT version: {peft.__version__}")
except ImportError as e:
    print(f"PEFT import error: {e}")

print("\n✅ If all versions are displayed above without errors, you can proceed to the next cell.")
print("❌ If you see import errors, please restart your kernel and re-run the installation cell.")

In [ ]:
# Fix torchvision::nms operator compatibility issue
print("🔧 Applying torchvision compatibility fixes...")

import os
import sys

# Method 1: Set environment variables to prevent torchvision conflicts
os.environ['TORCHVISION_DISABLE_VIDEO_BACKEND'] = '1'
os.environ['TORCH_DISABLE_JIT_COMPILATION'] = '1'

# Method 2: Import torch first and configure it
try:
    import torch
    
    # Disable JIT compilation which can cause operator registration issues
    torch.jit.set_fusion_strategy([('STATIC', 20), ('DYNAMIC', 20)])
    
    # Set torch to use CPU for meta operations if GPU causes issues
    if torch.cuda.is_available():
        torch.set_default_tensor_type('torch.FloatTensor')
    
    print("✅ Torch configured successfully")
    
except Exception as e:
    print(f"❌ Error configuring torch: {e}")

# Method 3: Import torchvision with fallback
try:
    # Try importing torchvision operations that might conflict
    import torchvision
    
    # Disable problematic torchvision operations
    if hasattr(torchvision, 'ops'):
        # Patch torchvision.ops.nms if it exists
        original_nms = getattr(torchvision.ops, 'nms', None)
        if original_nms:
            def safe_nms(*args, **kwargs):
                try:
                    return original_nms(*args, **kwargs)
                except Exception:
                    # Return empty tensor as fallback
                    return torch.empty(0, dtype=torch.long)
            torchvision.ops.nms = safe_nms
    
    print("✅ Torchvision patched successfully")
    
except Exception as e:
    print(f"⚠️ Torchvision patching warning: {e}")

# Method 4: Alternative library loading order
try:
    # Force reload transformers after torch configuration
    if 'transformers' in sys.modules:
        del sys.modules['transformers']
    
    # Import with specific backend settings
    os.environ['TRANSFORMERS_OFFLINE'] = '0'
    os.environ['HF_HUB_DISABLE_TELEMETRY'] = '1'
    
    print("✅ Environment prepared for transformers")
    
except Exception as e:
    print(f"⚠️ Environment setup warning: {e}")

print("🎯 Compatibility fixes applied. Proceeding with model loading...")

## Alternative Installation (If Above Fails)

If you continue to get the `torchvision::nms` error, try this minimal installation approach:

**Step 1: Uninstall conflicting packages**
```bash
pip uninstall torch torchvision torchaudio transformers -y
```

**Step 2: Install CPU-only versions (most compatible)**
```bash
pip install torch==2.0.1+cpu torchvision==0.15.2+cpu torchaudio==2.0.2+cpu --index-url https://download.pytorch.org/whl/cpu
pip install transformers==4.30.0 datasets==2.12.0 peft==0.4.0
```

**Step 3: Install remaining packages**
```bash
pip install accelerate matplotlib seaborn scipy pandas numpy
```

**Note:** This uses CPU-only PyTorch which is more stable but slower. If you need GPU support, try the original installation after this works.

In [ ]:
# CPU-only installation (run this if GPU installation fails)
# Uncomment the lines below if you want to use CPU-only versions

# !pip uninstall torch torchvision torchaudio transformers -y
# !pip install torch==2.0.1+cpu torchvision==0.15.2+cpu torchaudio==2.0.2+cpu --index-url https://download.pytorch.org/whl/cpu
# !pip install transformers==4.30.0 datasets==2.12.0 peft==0.4.0 accelerate
# !pip install matplotlib seaborn scipy pandas numpy

print("💡 If you're still getting errors, uncomment the lines above and run this cell")
print("This will install CPU-only versions which are more compatible but slower")

## Load Individual News Model and Tokenizer

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datasets import load_dataset
import json
import re
from scipy.stats import wilcoxon
from collections import defaultdict
import warnings

# Suppress warnings that might be caused by version mismatches
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

# Additional torchvision compatibility settings
import os
os.environ['TORCH_USE_CUDA_DSA'] = '1'
os.environ['TORCHVISION_DISABLE_VIDEO_BACKEND'] = '1'

print("🔧 Environment configured with compatibility settings...")

# Load the base model for individual news training
base_model_id = './Qwen3-8B'
adapter_path = './my-awesome-model_final_bitcoin-enhanced-prediction-dataset-with-local-comprehensive-news-v2/checkpoint-400'  # Adjust based on your checkpoint

print("📦 Loading base model and tokenizer...")
try:
    # Use CPU-only loading if GPU causes issues
    device_map = "auto"
    torch_dtype = torch.float16
    
    # Fallback to CPU if CUDA causes torchvision issues
    if not torch.cuda.is_available():
        device_map = "cpu"
        torch_dtype = torch.float32
        print("🔄 Using CPU mode for compatibility")
    
    # Load the base model and tokenizer with error handling
    model = AutoModelForCausalLM.from_pretrained(
        base_model_id,
        torch_dtype=torch_dtype,
        device_map=device_map,
        trust_remote_code=True,
        low_cpu_mem_usage=True,  # Help with memory issues
        use_safetensors=True if os.path.exists(os.path.join(base_model_id, "model.safetensors.index.json")) else False
    )

    tokenizer = AutoTokenizer.from_pretrained(
        base_model_id,
        trust_remote_code=True,
        use_fast=False  # Use slow tokenizer for better compatibility
    )

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    print("✅ Individual News Model loaded successfully!")
    print(f"📍 Model device: {next(model.parameters()).device}")
    print(f"📊 Model dtype: {next(model.parameters()).dtype}")
    
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("💡 Try the following solutions:")
    print("1. Run the CPU-only installation cell above")
    print("2. Restart your kernel and re-run all cells")
    print("3. Check if the model path './Qwen3_8B' exists")
    print("4. Use the absolute path to your model")
    
    # Try CPU fallback
    try:
        print("🔄 Attempting CPU fallback...")
        model = AutoModelForCausalLM.from_pretrained(
            base_model_id,
            torch_dtype=torch.float32,
            device_map="cpu",
            trust_remote_code=True,
            low_cpu_mem_usage=True
        )
        tokenizer = AutoTokenizer.from_pretrained(
            base_model_id,
            trust_remote_code=True,
            use_fast=False
        )
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        print("✅ CPU fallback successful!")
    except Exception as e2:
        print(f"❌ CPU fallback also failed: {e2}")
        raise

## Load LoRA Adapter from Individual News Training

In [ ]:
# Load the LoRA adapter from the individual news training (checkpoint 200)

print("📦 Loading LoRA adapter...")
try:
    # Load the model with LoRA adapter with enhanced error handling
    model = PeftModel.from_pretrained(
        model, 
        adapter_path,
        is_trainable=False,  # Set to False for inference
        torch_dtype=next(model.parameters()).dtype  # Match base model dtype
    )
    model.eval()

    print("✅ Individual News Model with adapter (checkpoint-200) loaded successfully!")
    print(f"📍 Adapter loaded from: {adapter_path}")
    
except FileNotFoundError as e:
    print(f"❌ Adapter path not found: {e}")
    print("💡 Possible solutions:")
    print("1. Check if the adapter path exists: './qwen_bitcoin_chat_fast_more_longer/checkpoint-200'")
    print("2. Verify that the checkpoint contains the required adapter files")
    print("3. Try using a different checkpoint if available")
    
    # List available checkpoints if the directory exists
    import os
    base_dir = './qwen_bitcoin_chat_fast_more_longer'
    if os.path.exists(base_dir):
        try:
            items = os.listdir(base_dir)
            checkpoints = [d for d in items if d.startswith('checkpoint-') and os.path.isdir(os.path.join(base_dir, d))]
            if checkpoints:
                checkpoints.sort(key=lambda x: int(x.split('-')[1]) if x.split('-')[1].isdigit() else 0)
                print(f"📁 Available checkpoints: {checkpoints}")
                
                # Try the latest checkpoint automatically
                latest_checkpoint = checkpoints[-1]
                latest_path = os.path.join(base_dir, latest_checkpoint)
                print(f"🔄 Attempting to load latest checkpoint: {latest_checkpoint}")
                
                try:
                    model = PeftModel.from_pretrained(
                        model, 
                        latest_path,
                        is_trainable=False,
                        torch_dtype=next(model.parameters()).dtype
                    )
                    model.eval()
                    print(f"✅ Successfully loaded latest checkpoint: {latest_checkpoint}")
                except Exception as e2:
                    print(f"❌ Failed to load latest checkpoint: {e2}")
                    raise
            else:
                print("❌ No checkpoint directories found in the base directory")
                raise
        except Exception as list_error:
            print(f"❌ Error listing directory contents: {list_error}")
            raise
    else:
        print(f"❌ Base directory '{base_dir}' does not exist")
        
        # Check for alternative paths
        alternative_paths = [
            './checkpoint-200',
            './checkpoints/checkpoint-200',
            './model_outputs/checkpoint-200',
            '../checkpoint-200'
        ]
        
        for alt_path in alternative_paths:
            if os.path.exists(alt_path):
                print(f"🔍 Found alternative path: {alt_path}")
                try:
                    model = PeftModel.from_pretrained(
                        model, 
                        alt_path,
                        is_trainable=False,
                        torch_dtype=next(model.parameters()).dtype
                    )
                    model.eval()
                    print(f"✅ Successfully loaded adapter from: {alt_path}")
                    break
                except Exception as e3:
                    print(f"❌ Failed to load from {alt_path}: {e3}")
                    continue
        else:
            print("❌ No valid adapter paths found")
            raise
    
except Exception as e:
    print(f"❌ Error loading LoRA adapter: {e}")
    print("💡 Alternative solutions:")
    print("1. Ensure the checkpoint was saved properly during training")
    print("2. Check file permissions on the checkpoint directory")
    print("3. Try loading without adapter for base model evaluation")
    
    # Option to continue without adapter
    response = input("Continue without adapter? (y/n): ").lower().strip()
    if response == 'y':
        print("⚠️ Continuing with base model only (no adapter loaded)")
    else:
        raise

## Load Test Data from Individual News Dataset

In [ ]:
# Load the individual news dataset
test_dataset = load_dataset('tahamajs/bitcoin-investment-advisory-dataset', split='test')
print(f"Loaded {len(test_dataset)} test samples")
print("Sample test data:")
print(test_dataset[0])

## Utility Functions for Analysis

In [ ]:
def extract_prices_from_text(text):
    """Extract price predictions from model output"""
    # Look for patterns like numbers separated by commas
    price_pattern = r'(\d+(?:\.\d+)?(?:,\s*\d+(?:\.\d+)?)*)'  
    matches = re.findall(price_pattern, text)
    
    if matches:
        # Take the first match and split by comma
        prices_str = matches[0]
        try:
            prices = [float(p.strip()) for p in prices_str.split(',')]
            return prices
        except:
            return []
    return []

def calculate_metrics(predictions, ground_truth):
    """Calculate evaluation metrics"""
    if len(predictions) != len(ground_truth):
        return None
    
    predictions = np.array(predictions)
    ground_truth = np.array(ground_truth)
    
    mse = np.mean((predictions - ground_truth) ** 2)
    mae = np.mean(np.abs(predictions - ground_truth))
    mape = np.mean(np.abs((ground_truth - predictions) / ground_truth)) * 100
    
    return {
        'MSE': mse,
        'MAE': mae,
        'MAPE': mape,
        'RMSE': np.sqrt(mse)
    }

def analyze_text_quality(generated_text, expected_text):
    """Analyze text quality metrics beyond numerical predictions"""
    # Text length analysis
    gen_length = len(generated_text.split())
    exp_length = len(expected_text.split())
    length_ratio = gen_length / exp_length if exp_length > 0 else 0
    
    # Word overlap analysis
    gen_words = set(generated_text.lower().split())
    exp_words = set(expected_text.lower().split())
    
    intersection = gen_words.intersection(exp_words)
    union = gen_words.union(exp_words)
    
    jaccard_similarity = len(intersection) / len(union) if len(union) > 0 else 0
    
    # Check for key financial terms
    financial_terms = ['price', 'market', 'trend', 'analysis', 'prediction', 'forecast', 
                      'bitcoin', 'btc', 'increase', 'decrease', 'bullish', 'bearish',
                      'investment', 'trading', 'volatility', 'support', 'resistance']
    
    gen_financial_terms = sum(1 for term in financial_terms if term in generated_text.lower())
    exp_financial_terms = sum(1 for term in financial_terms if term in expected_text.lower())
    
    return {
        'length_ratio': length_ratio,
        'jaccard_similarity': jaccard_similarity,
        'generated_length': gen_length,
        'expected_length': exp_length,
        'generated_financial_terms': gen_financial_terms,
        'expected_financial_terms': exp_financial_terms,
        'financial_term_coverage': gen_financial_terms / max(exp_financial_terms, 1)
    }

def format_input(example):
    """Format input for the individual news model"""
    instruction = example.get('instruction', '')
    user_input = example.get('input', '')
    messages = [
        {'role': 'system', 'content': instruction},
        {'role': 'user', 'content': user_input}
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

print("Utility functions loaded!")

## Comprehensive Model Evaluation

In [ ]:
# Run comprehensive evaluation
results = []
text_quality_results = []
total_samples = min(100, len(test_dataset))  # Test on 100 samples or all if less

print(f"Running comprehensive evaluation on {total_samples} samples...")

for i in range(total_samples):
    test_example = test_dataset[i]
    test_text = format_input(test_example)
    
    inputs = tokenizer(test_text, return_tensors='pt', truncation=True, max_length=2048)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=False,  # Use greedy decoding for consistency
            pad_token_id=tokenizer.eos_token_id
        )
    
    generated_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    
    # Extract predictions and ground truth
    predicted_prices = extract_prices_from_text(generated_text)
    actual_output = test_example.get('output', '')
    actual_prices = extract_prices_from_text(actual_output)
    
    # Analyze text quality
    text_quality = analyze_text_quality(generated_text, actual_output)
    text_quality_results.append(text_quality)
    
    if predicted_prices and actual_prices:
        # Truncate to minimum length for fair comparison
        min_len = min(len(predicted_prices), len(actual_prices))
        if min_len > 0:
            pred_truncated = predicted_prices[:min_len]
            actual_truncated = actual_prices[:min_len]
            
            metrics = calculate_metrics(pred_truncated, actual_truncated)
            if metrics:
                results.append({
                    'sample_id': i,
                    'predicted': pred_truncated,
                    'actual': actual_truncated,
                    'metrics': metrics,
                    'text_quality': text_quality,
                    'generated_text': generated_text,
                    'expected_text': actual_output
                })
    
    if (i + 1) % 20 == 0:
        print(f"Processed {i + 1}/{total_samples} samples...")

print(f"\nEvaluation completed! Analyzed {len(results)} valid samples out of {total_samples}")

## Detailed Results Analysis

In [ ]:
if results:
    # Calculate numerical performance metrics
    all_mse = [r['metrics']['MSE'] for r in results]
    all_mae = [r['metrics']['MAE'] for r in results]
    all_mape = [r['metrics']['MAPE'] for r in results]
    all_rmse = [r['metrics']['RMSE'] for r in results]
    
    # Calculate text quality metrics
    all_length_ratios = [r['text_quality']['length_ratio'] for r in results]
    all_jaccard_sim = [r['text_quality']['jaccard_similarity'] for r in results]
    all_financial_coverage = [r['text_quality']['financial_term_coverage'] for r in results]
    
    print("=" * 80)
    print("🎯 INDIVIDUAL NEWS MODEL - COMPREHENSIVE ANALYSIS")
    print("=" * 80)
    
    print(f"\n📊 NUMERICAL PERFORMANCE METRICS:")
    print(f"Mean Squared Error (MSE): {np.mean(all_mse):.4f} ± {np.std(all_mse):.4f}")
    print(f"Mean Absolute Error (MAE): {np.mean(all_mae):.4f} ± {np.std(all_mae):.4f}")
    print(f"Root Mean Squared Error (RMSE): {np.mean(all_rmse):.4f} ± {np.std(all_rmse):.4f}")
    print(f"Mean Absolute Percentage Error (MAPE): {np.mean(all_mape):.2f}% ± {np.std(all_mape):.2f}%")
    
    print(f"\n📝 TEXT QUALITY METRICS:")
    print(f"Average Response Length Ratio: {np.mean(all_length_ratios):.2f} ± {np.std(all_length_ratios):.2f}")
    print(f"Average Jaccard Similarity: {np.mean(all_jaccard_sim):.3f} ± {np.std(all_jaccard_sim):.3f}")
    print(f"Financial Term Coverage: {np.mean(all_financial_coverage):.2f} ± {np.std(all_financial_coverage):.2f}")
    
    print(f"\n📋 DETAILED TEXT ANALYSIS:")
    avg_gen_length = np.mean([r['text_quality']['generated_length'] for r in results])
    avg_exp_length = np.mean([r['text_quality']['expected_length'] for r in results])
    avg_gen_fin_terms = np.mean([r['text_quality']['generated_financial_terms'] for r in results])
    avg_exp_fin_terms = np.mean([r['text_quality']['expected_financial_terms'] for r in results])
    
    print(f"Average Generated Response Length: {avg_gen_length:.1f} words")
    print(f"Average Expected Response Length: {avg_exp_length:.1f} words")
    print(f"Average Financial Terms in Generated: {avg_gen_fin_terms:.1f}")
    print(f"Average Financial Terms in Expected: {avg_exp_fin_terms:.1f}")
    
    # Median metrics for robustness
    print(f"\n📊 MEDIAN PERFORMANCE METRICS (Robust):")
    print(f"Median MSE: {np.median(all_mse):.4f}")
    print(f"Median MAE: {np.median(all_mae):.4f}")
    print(f"Median RMSE: {np.median(all_rmse):.4f}")
    print(f"Median MAPE: {np.median(all_mape):.2f}%")
    print(f"Median Jaccard Similarity: {np.median(all_jaccard_sim):.3f}")
    
    # Sample outputs for qualitative analysis
    print(f"\n🔍 SAMPLE OUTPUTS FOR QUALITATIVE ANALYSIS:")
    for i, result in enumerate(results[:3]):
        print(f"\n--- Sample {i+1} ---")
        print(f"Predicted Prices: {result['predicted']}")
        print(f"Actual Prices:    {result['actual']}")
        print(f"MAE: {result['metrics']['MAE']:.4f}, MAPE: {result['metrics']['MAPE']:.2f}%")
        print(f"Text Similarity: {result['text_quality']['jaccard_similarity']:.3f}")
        print(f"Generated Text: {result['generated_text'][:200]}...")
        print(f"Expected Text:  {result['expected_text'][:200]}...")
        print("-" * 60)

else:
    print("❌ No valid results found for analysis")

## Load Base Qwen Model for Comparison

In [ ]:
# Load base Qwen model for comparison
print("Loading base Qwen model for comparison...")

base_qwen_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-8B-Instruct",
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

base_qwen_tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen2.5-8B-Instruct",
    trust_remote_code=True
)

if base_qwen_tokenizer.pad_token is None:
    base_qwen_tokenizer.pad_token = base_qwen_tokenizer.eos_token

print("Base Qwen model loaded successfully!")

In [ ]:
# Evaluate base model
def format_input_for_base_model(example):
    """Format input for base Qwen model with bitcoin prediction task"""
    instruction = example.get('instruction', '')
    user_input = example.get('input', '')
    
    bitcoin_instruction = """You are a Bitcoin investment advisor. Based on the provided market data and news, predict the next 10 days of Bitcoin prices. Provide your predictions as comma-separated numbers."""
    
    messages = [
        {'role': 'system', 'content': bitcoin_instruction},
        {'role': 'user', 'content': f"{instruction}\n\n{user_input}\n\nPlease provide 10 Bitcoin price predictions for the next 10 days, separated by commas."}
    ]
    return base_qwen_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Run evaluation on base model
base_results = []
base_text_quality_results = []
comparison_samples = min(50, len(results))  # Use subset for comparison

print(f"Evaluating base Qwen model on {comparison_samples} samples...")

for i in range(comparison_samples):
    test_example = test_dataset[i]
    test_text = format_input_for_base_model(test_example)
    
    inputs = base_qwen_tokenizer(test_text, return_tensors='pt', truncation=True, max_length=2048)
    inputs = {k: v.to(base_qwen_model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = base_qwen_model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=False,
            pad_token_id=base_qwen_tokenizer.eos_token_id
        )
    
    generated_text = base_qwen_tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    
    predicted_prices = extract_prices_from_text(generated_text)
    actual_output = test_example.get('output', '')
    actual_prices = extract_prices_from_text(actual_output)
    
    # Analyze text quality for base model
    text_quality = analyze_text_quality(generated_text, actual_output)
    base_text_quality_results.append(text_quality)
    
    if predicted_prices and actual_prices:
        min_len = min(len(predicted_prices), len(actual_prices))
        if min_len > 0:
            pred_truncated = predicted_prices[:min_len]
            actual_truncated = actual_prices[:min_len]
            
            metrics = calculate_metrics(pred_truncated, actual_truncated)
            if metrics:
                base_results.append({
                    'sample_id': i,
                    'predicted': pred_truncated,
                    'actual': actual_truncated,
                    'metrics': metrics,
                    'text_quality': text_quality,
                    'generated_text': generated_text,
                    'expected_text': actual_output
                })
    
    if (i + 1) % 10 == 0:
        print(f"Processed {i + 1}/{comparison_samples} samples...")

print(f"\nBase model evaluation completed! Analyzed {len(base_results)} valid samples")

## Comprehensive Model Comparison

In [ ]:
if base_results and results:
    # Extract metrics from results for proper comparison
    # Individual News Model metrics
    ind_mse = [r['metrics']['MSE'] for r in results]
    ind_mae = [r['metrics']['MAE'] for r in results] 
    ind_rmse = [r['metrics']['RMSE'] for r in results]
    ind_mape = [r['metrics']['MAPE'] for r in results]
    ind_jaccard = [r['text_quality']['jaccard_similarity'] for r in results]
    ind_length_ratio = [r['text_quality']['length_ratio'] for r in results]
    ind_financial_coverage = [r['text_quality']['financial_term_coverage'] for r in results]
    
    # Base Model metrics
    base_mse = [r['metrics']['MSE'] for r in base_results]
    base_mae = [r['metrics']['MAE'] for r in base_results]
    base_rmse = [r['metrics']['RMSE'] for r in base_results]
    base_mape = [r['metrics']['MAPE'] for r in base_results]
    base_jaccard = [r['text_quality']['jaccard_similarity'] for r in base_results]
    base_length_ratio = [r['text_quality']['length_ratio'] for r in base_results]
    base_financial_coverage = [r['text_quality']['financial_term_coverage'] for r in base_results]
    
    # Statistical significance testing
    from scipy.stats import wilcoxon
    
    # Test for significant differences (using matched samples)
    min_samples = min(len(results), len(base_results))
    if min_samples >= 5:  # Minimum for statistical testing
        mse_stat, mse_pvalue = wilcoxon(ind_mse[:min_samples], base_mse[:min_samples])
        mae_stat, mae_pvalue = wilcoxon(ind_mae[:min_samples], base_mae[:min_samples])
        jaccard_stat, jaccard_pvalue = wilcoxon(ind_jaccard[:min_samples], base_jaccard[:min_samples])
        
        print(f"\n🔬 STATISTICAL SIGNIFICANCE TESTS:")
        print(f"MSE difference p-value: {mse_pvalue:.4f} ({'Significant' if mse_pvalue < 0.05 else 'Not significant'})")
        print(f"MAE difference p-value: {mae_pvalue:.4f} ({'Significant' if mae_pvalue < 0.05 else 'Not significant'})")
        print(f"Jaccard similarity p-value: {jaccard_pvalue:.4f} ({'Significant' if jaccard_pvalue < 0.05 else 'Not significant'})")
    
    # Save comprehensive results
    individual_news_comprehensive_results = {
        'individual_news_model_results': {
            'model_id': 'tahamajs/my-awesome-model_final_bitcoin-individual-news-dataset',
            'adapter_path': './qwen_bitcoin_chat_fast_more_longer/checkpoint-200',
            'dataset': 'bitcoin-investment-advisory-dataset',
            'training_config': 'longer_training_4_epochs_checkpoint_200',
            'samples_analyzed': len(results),
            'numerical_metrics': {
                'mean_mse': float(np.mean(ind_mse)),
                'mean_mae': float(np.mean(ind_mae)),
                'mean_rmse': float(np.mean(ind_rmse)),
                'mean_mape': float(np.mean(ind_mape))
            },
            'text_quality_metrics': {
                'mean_jaccard_similarity': float(np.mean(ind_jaccard)),
                'mean_length_ratio': float(np.mean(ind_length_ratio)),
                'mean_financial_coverage': float(np.mean(ind_financial_coverage))
            }
        },
        'base_model_results': {
            'model_id': 'Qwen/Qwen2.5-8B-Instruct',
            'samples_analyzed': len(base_results),
            'numerical_metrics': {
                'mean_mse': float(np.mean(base_mse)),
                'mean_mae': float(np.mean(base_mae)),
                'mean_rmse': float(np.mean(base_rmse)),
                'mean_mape': float(np.mean(base_mape))
            },
            'text_quality_metrics': {
                'mean_jaccard_similarity': float(np.mean(base_jaccard)),
                'mean_length_ratio': float(np.mean(base_length_ratio)),
                'mean_financial_coverage': float(np.mean(base_financial_coverage))
            }
        },
        'detailed_individual_results': results,
        'detailed_base_results': base_results
    }
    
    with open('individual_news_model_comprehensive_analysis.json', 'w') as f:
        json.dump(individual_news_comprehensive_results, f, indent=2)
    
    print(f"\n💾 Comprehensive analysis results saved to 'individual_news_model_comprehensive_analysis.json'")
    
    # Research paper summary
    print(f"\n📋 RESEARCH PAPER SUMMARY - INDIVIDUAL NEWS MODEL:")
    print("=" * 70)
    mse_improvement = (np.mean(base_mse) - np.mean(ind_mse)) / np.mean(base_mse) * 100
    mae_improvement = (np.mean(base_mae) - np.mean(ind_mae)) / np.mean(base_mae) * 100
    jaccard_improvement = (np.mean(ind_jaccard) - np.mean(base_jaccard)) / np.mean(base_jaccard) * 100
    
    print(f"Individual News Model Performance:")
    print(f"• MSE improved by {mse_improvement:.2f}%")
    print(f"• MAE improved by {mae_improvement:.2f}%")
    print(f"• Text similarity improved by {jaccard_improvement:.2f}%")
    print(f"• Mean Jaccard Similarity: {np.mean(ind_jaccard):.3f}")
    print(f"• Financial term coverage: {np.mean(ind_financial_coverage):.2f}")
    print(f"• Model: tahamajs/my-awesome-model_final_bitcoin-individual-news-dataset")
    print(f"• Checkpoint: 200 (4 epochs with longer parameters)")

else:
    print("❌ No valid results found for model comparison")

## Visualization of Results

In [ ]:
if base_results and results:
    # Create comprehensive visualization
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Individual News Model vs Base Qwen - Comprehensive Analysis', fontsize=16, fontweight='bold')
    
    # MSE comparison
    axes[0,0].boxplot([base_mse, ind_mse], labels=['Base Qwen', 'Individual News'])
    axes[0,0].set_title('Mean Squared Error', fontweight='bold')
    axes[0,0].set_ylabel('MSE')
    axes[0,0].grid(True, alpha=0.3)
    
    # MAE comparison
    axes[0,1].boxplot([base_mae, ind_mae], labels=['Base Qwen', 'Individual News'])
    axes[0,1].set_title('Mean Absolute Error', fontweight='bold')
    axes[0,1].set_ylabel('MAE')
    axes[0,1].grid(True, alpha=0.3)
    
    # MAPE comparison
    axes[0,2].boxplot([base_mape, ind_mape], labels=['Base Qwen', 'Individual News'])
    axes[0,2].set_title('Mean Absolute Percentage Error', fontweight='bold')
    axes[0,2].set_ylabel('MAPE (%)')
    axes[0,2].grid(True, alpha=0.3)
    
    # Jaccard similarity comparison
    axes[1,0].boxplot([base_jaccard, ind_jaccard], labels=['Base Qwen', 'Individual News'])
    axes[1,0].set_title('Text Similarity (Jaccard)', fontweight='bold')
    axes[1,0].set_ylabel('Jaccard Similarity')
    axes[1,0].grid(True, alpha=0.3)
    
    # Length ratio comparison
    axes[1,1].boxplot([base_length_ratio, ind_length_ratio], labels=['Base Qwen', 'Individual News'])
    axes[1,1].set_title('Response Length Ratio', fontweight='bold')
    axes[1,1].set_ylabel('Length Ratio')
    axes[1,1].grid(True, alpha=0.3)
    
    # Improvement bar chart
    metrics = ['MSE', 'MAE', 'MAPE', 'Text Sim', 'Length', 'Fin Terms']
    improvements = [
        (np.mean(base_mse) - np.mean(ind_mse)) / np.mean(base_mse) * 100,
        (np.mean(base_mae) - np.mean(ind_mae)) / np.mean(base_mae) * 100,
        (np.mean(base_mape) - np.mean(ind_mape)) / np.mean(base_mape) * 100,
        (np.mean(ind_jaccard) - np.mean(base_jaccard)) / np.mean(base_jaccard) * 100,
        (np.mean(ind_length_ratio) - np.mean(base_length_ratio)) / np.mean(base_length_ratio) * 100,
        (np.mean(ind_financial_coverage) - np.mean(base_financial_coverage)) / np.mean(base_financial_coverage) * 100
    ]
    
    colors = ['green' if imp > 0 else 'red' for imp in improvements]
    bars = axes[1,2].bar(metrics, improvements, color=colors, alpha=0.7)
    
    # Add value labels on bars
    for bar, imp in zip(bars, improvements):
        height = bar.get_height()
        axes[1,2].text(bar.get_x() + bar.get_width()/2., height,
                       f'{imp:.1f}%', ha='center', va='bottom' if height > 0 else 'top',
                       fontweight='bold', fontsize=8)
    
    axes[1,2].set_title('Performance Improvements', fontweight='bold')
    axes[1,2].set_ylabel('Improvement (%)')
    axes[1,2].axhline(y=0, color='black', linestyle='-', alpha=0.3)
    axes[1,2].grid(True, alpha=0.3)
    axes[1,2].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig('individual_news_model_comprehensive_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("📊 Comprehensive analysis visualization saved as 'individual_news_model_comprehensive_analysis.png'")

else:
    print("❌ Cannot create visualizations - insufficient data")